In [33]:
!pip install -q langchain faiss-gpu sentence-transformers==2.2.2

In [34]:
import os
import pandas as pd

from tqdm import tqdm
from google.colab import drive
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [35]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/wallpaper-defects-qa/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/wallpaper-defects-qa


# Documents

In [36]:
document_name = 'test_document.csv'

In [37]:
loader = CSVLoader(file_path=f'data/documents/{document_name}')
data = loader.load()

# Embedding model

In [47]:
embedding_model_name = 'distiluse-base-multilingual-cased-v1'
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': False}

In [48]:
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Index

In [49]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [50]:
vectorstore.save_local(f'faiss_index/faiss_index_{os.path.basename(document_name)}')

In [51]:
# vectorstore = FAISS.load_local("faiss_index", embeddings)

# Retriever

In [53]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LLM

In [54]:
model_id = "hongzoh/wdqa-v1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(0)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
    )
hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Chain

In [55]:
template = """다음 내용을 기반으로 질문에 대답해주세요:

{context}

질문: {question}
"""

In [56]:
prompt = ChatPromptTemplate.from_template(template)

In [57]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [58]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | hf
    | StrOutputParser()
)

# Test data

In [59]:
test_file_name = 'test.csv'
test_df = pd.read_csv('data/open/' + test_file_name)
test_df.head()

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?


In [60]:
q_list = test_df['질문'].values.tolist()

# Test

# Output

In [61]:
outputs = []
for q in tqdm(q_list[:3]):
    output = chain.invoke(q)
    outputs.append(output)
    print(output)

 33%|███▎      | 1/3 [00:06<00:13,  6.98s/it]

: 방청 페인트의 종류는 주로 유광과 무광, 그리고 방청 코팅이 있습니다. 유광페인트는 주로 외부에서 사용되며 방청 효과가 있고 광택이 납니다. 반면 무광페인트는 주로 내부에서 사용되며 고농축 도료가 장점이 될 수 있으나 유광페인트보다 빨리 낡고 광택이 없다는 단점이 있습니다. 원목사이딩은 편백나무로 만들어지는 원재료이며 수공정이 포함되어 있어 다른 구조물보다 시공이 더 어렵고 가격이 비쌉니다. 또한 원목사이딩의 외관이 무광이라 유광페인트와 대비할 때 주변이 더


 67%|██████▋   | 2/3 [00:13<00:06,  6.44s/it]

답변: 도배지는 표면이 넓고 평활하고 평면적이며 건조 수축이 있는 재료입니다. 또한 습기가 많은 환경에서 적합하지 않고 표면의 강도가 낮아 습기가 차면 곰팡이가 발생할 수 있습니다. 1. 공간의 습도가 조절되지 않아 도배지의 표면에 습기가 많으면 곰팡이 번식으로 이어질 수 있습니다. 공간의 습도를 확인하여 환기량을 늘리는 등의 대응이 필요합니다. 2. 실내 온도가 25도 이상인 경우 도배지에 곰팡이가 발생할 수 있습니다. 온도가 높으면 습기가 계속 축적되기 때문에, 적절한 환기와 채광으로 실내 온도를 조절하는 것이 좋습니다. 실내 온도는 15~20도를 유지


100%|██████████| 3/3 [00:18<00:00,  6.02s/it]

​답변: 큐블럭의 단점은 보철을 사용한 제품이므로 충격에 약하고 표면이 오목하거나 함몰될 가능성이 있다는 점입니다. 또한 표면에 도색작업을 하지 않고는 미관상 좋지 않은 모습을 보이기도 합니다. 하지만 외부 온도 변화에 의한 영향을 비교적 덜 받고 표면이 오목해지거나 함몰되는 문제를 보완하는 보강 처리를 하는 등 추가적인 장점이 있습니다. 또한 압축법 단열판은 발포 플라스틱을 사용하여 제조되기 때문에 접착제를 사용하지 않고도 벽이나 천장에 부착할 수 있는 장점이 있습니다. 또한 압출
